# Back-up Tabla 1 and Tabla 2 (extraction and data clean-up

## Común

In [ ]:
# 1.
def drop_nan_rows(df):
    df = df.dropna(how='all')
    return df

# 2. 
def drop_nan_columns(df):
    return df.dropna(axis=1, how='all')

# 3.
def swap_first_second_row(df):
    temp = df.iloc[0, 0]
    df.iloc[0, 0] = df.iloc[1, 0]
    df.iloc[1, 0] = temp

    temp = df.iloc[0, -1]
    df.iloc[0, -1] = df.iloc[1, -1]
    df.iloc[1, -1] = temp
    return df

# 4. 
def reset_index(df):
    df.reset_index(drop=True, inplace=True)
    return df

# 5.
def remove_digit_slash(df):
    # Aplica la función de reemplazo a la primera columna y a las dos últimas columnas
    df.iloc[:, [0, -2, -1]] = df.iloc[:, [0, -2, -1]].apply(lambda x: x.str.replace(r'\d+/', '', regex=True))
    return df

# 6. AUX (ROBUSTO)

def separate_text_digits(df):
    for index, row in df.iterrows():
        if any(char.isdigit() for char in str(row.iloc[-2])) and any(char.isalpha() for char in str(row.iloc[-2])):
            if pd.isnull(row.iloc[-1]):
                df.loc[index, df.columns[-1]] = ''.join(filter(lambda x: x.isalpha() or x == ' ', str(row.iloc[-2])))
                df.loc[index, df.columns[-2]] = ''.join(filter(lambda x: not (x.isalpha() or x == ' '), str(row.iloc[-2])))
            
            # Check if comma or dot is used as decimal separator
            if ',' in str(row.iloc[-2]):
                split_values = str(row.iloc[-2]).split(',')
            elif '.' in str(row.iloc[-2]):
                split_values = str(row.iloc[-2]).split('.')
            else:
                # If neither comma nor dot found, assume no decimal part
                split_values = [str(row.iloc[-2]), '']
                
            cleaned_integer = ''.join(filter(lambda x: x.isdigit() or x == '-', split_values[0]))
            cleaned_decimal = ''.join(filter(lambda x: x.isdigit(), split_values[1]))
            if cleaned_decimal:
                # Use comma as decimal separator
                cleaned_numeric = cleaned_integer + ',' + cleaned_decimal
            else:
                cleaned_numeric = cleaned_integer
            df.loc[index, df.columns[-2]] = cleaned_numeric
    return df


# 7. 
def extract_years(df):
    year_columns = [col for col in df.columns if re.match(r'\b\d{4}\b', col)]
    #print("Años (4 dígitos) extraídos:")
    #print(year_columns)
    return year_columns

# 8. 
def first_row_columns(df):
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])
    return df

# 9.
def clean_columns_values(df):
    df.columns = df.columns.str.lower()
    # Only normalize string column names
    df.columns = [unicodedata.normalize('NFKD', col).encode('ASCII', 'ignore').decode('utf-8') if isinstance(col, str) else col for col in df.columns]
    df.columns = df.columns.str.replace(' ', '_').str.replace('ano', 'year').str.replace('-', '_')
    
    text_columns = df.select_dtypes(include='object').columns
    for col in df.columns:
        df.loc[:, col] = df[col].apply(lambda x: remove_tildes(x) if isinstance(x, str) else x)
        df.loc[:, col] = df[col].apply(lambda x: str(x).replace(',', '.') if isinstance(x, (int, float, str)) else x)
    df.loc[:, 'sectores_economicos'] = df['sectores_economicos'].str.lower()
    df.loc[:, 'economic_sectors'] = df['economic_sectors'].str.lower()
    df.loc[:, 'sectores_economicos'] = df['sectores_economicos'].apply(remove_rare_characters)
    df.loc[:, 'economic_sectors'] = df['economic_sectors'].apply(remove_rare_characters)
    return df

# 10.
def convert_float(df):
    excluded_columns = ['sectores_economicos', 'economic_sectors']
    columns_to_convert = [col for col in df.columns if col not in excluded_columns]
    df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')
    return df

# 11.
def relocate_last_column(df):
    last_column = df.pop(df.columns[-1])
    df.insert(1, last_column.name, last_column)
    return df

### Exclusiva Tabla 1

In [ ]:
# 1.
def relocate_last_columns(df): # It is not the same as relocate_last_column
    if not pd.isna(df.iloc[1, -1]):
        # Create a new column with NaN
        new_column = 'col_' + ''.join(map(str, np.random.randint(1, 5, size=1)))
        df[new_column] = np.nan
        
        # Get 'ECONOMIC SECTORS' and relocate
        insert_value_1 = df.iloc[0, -2]
        # Convert the value to string before assignment
        insert_value_1 = str(insert_value_1)
        # Ensure the dtype of the last column is object (string) to accommodate string values
        df.iloc[:, -1] = df.iloc[:, -1].astype('object')
        df.iloc[0, -1] = insert_value_1
        
        # NaN first obs
        df.iloc[0,-2] = np.nan
    return df

# 2.
def get_months_sublist_list(df, year_columns):
    first_row = df.iloc[0]
    # Initialize the list of sublists
    months_sublist_list = []

    # Initialize the current sublist
    months_sublist = []

    # Iterate over the elements of the first row
    for item in first_row:
        # Check if the item meets the requirements
        if len(str(item)) == 3:
            months_sublist.append(item)
        elif '-' in item or str(item) == 'year':
            months_sublist.append(item)
            months_sublist_list.append(months_sublist)
            months_sublist = []

    # Add the last sublist if it's not empty
    if months_sublist:
        months_sublist_list.append(months_sublist)

    new_elements = []

    # Check if year_columns is not empty
    if year_columns:
        for i, year in enumerate(year_columns):
            # Check if index i is valid for quarters_sublist_list
            if i < len(months_sublist_list):
                for element in months_sublist_list[i]:
                    new_elements.append(f"{year}_{element}")
                    
    two_first_elements = df.iloc[0][:2].tolist()

    # Ensure that the two_first_elements are added if they are not in new_elements
    for index in range(len(two_first_elements) - 1, -1, -1):
        if two_first_elements[index] not in new_elements:
            new_elements.insert(0, two_first_elements[index])

    # Ensure that the length of new_elements matches the number of columns in df
    while len(new_elements) < len(df.columns):
        new_elements.append(None)

    temp_df = pd.DataFrame([new_elements], columns=df.columns)
    df.iloc[0] = temp_df.iloc[0]

    return df

# 3.
def find_year_column(df):
    # List to store the found years
    found_years = []

    # Iterating over the column names of the DataFrame
    for column in df.columns:
        # Checking if the column name is a year (4 digits)
        if column.isdigit() and len(column) == 4:
            found_years.append(column)

    # If more than one year is found, do nothing
    if len(found_years) > 1:
        pass
    # If exactly one year is found, implement additional code
    elif len(found_years) == 1:
        # Getting the name of the found year
        year_name = found_years[0]
        print("The name of the column representing the year is:", year_name)

        # Getting the first row of the DataFrame
        first_row = df.iloc[0]

        # Searching for the first column containing the word "year" or some hyphen-separated expression
        column_contains_year = first_row[first_row.astype(str).str.contains(r'\byear\b')]

        if not column_contains_year.empty:
            # Getting the name of the first column containing 'year' or some hyphen-separated expression in the first row
            column_contains_year_name = column_contains_year.index[0]
            print("The name of the first column containing 'year' or some hyphen-separated expression in the first row is:", column_contains_year_name)

            # Getting the indices of the columns
            column_contains_year_index = df.columns.get_loc(column_contains_year_name)
            year_name_index = df.columns.get_loc(year_name)
            print("The index of the column containing 'year' is:", column_contains_year_index)
            print("The index of the column representing the year is:", year_name_index)

            # Checking if the column representing the year is to the right or to the left of column_contains_year
            if column_contains_year_index < year_name_index:
                print("The year column is to the right of the column containing 'year'.")
                # Adding one to the year
                new_year = str(int(year_name) - 1)
                # Renaming the column containing 'year' with the new year
                df.rename(columns={column_contains_year_name: new_year}, inplace=True)
                print(f"The column containing 'year' is now named '{new_year}'.")
            elif column_contains_year_index > year_name_index:
                print("The year column is to the left of the column containing 'year'.")
                # Subtracting one from the year
                new_year = str(int(year_name) + 1)
                # Renaming the year column with the new year
                df.rename(columns={column_contains_year_name: new_year}, inplace=True)
                print(f"The column containing 'year' is now named '{new_year}'.")
            else:
                print("The year column is in the same position as the column containing 'year'.")
        else:
            print("No columns containing 'year' were found in the first row.")
    # If no year is found, print a message
    else:
        print("No years were found in the column names.")
    
    return df

# 4.
def exchange_values(df):
    # Verificar si hay al menos dos columnas en el DataFrame
    if len(df.columns) < 2:
        print("El DataFrame tiene menos de dos columnas. No se pueden intercambiar valores.")
        return df

    # Verificar si hay valores NaN en la última columna
    if df.iloc[:, -1].isnull().any():
        # Obtener índice de filas con NaN en la última columna
        last_column_rows_nan = df[df.iloc[:, -1].isnull()].index

        # Iterar sobre las filas con NaN en la última columna
        for idx in last_column_rows_nan:
            # Verificar si el índice está dentro del rango de las columnas
            if -2 >= -len(df.columns):
                # Intercambiar los valores de la última columna y la penúltima columna
                df.iloc[idx, -1], df.iloc[idx, -2] = df.iloc[idx, -2], df.iloc[idx, -1]
            else:
                print(f"Índice fuera de rango para la fila {idx}. No se pueden intercambiar valores.")

    return df

# 5.
def replace_var_perc_first_column(df):
    # Regular expression to search for "Var. %" or "Var.%"
    regex = re.compile(r'Var\. ?%')

    # Iterate over the rows of the dataframe
    for index, row in df.iterrows():
        # Convert the value in the first column to a string
        value = str(row.iloc[0])

        # Check if the value matches the regular expression
        if regex.search(value):
            # Replace only the characters that match the regular expression
            df.at[index, df.columns[0]] = regex.sub("variacion porcentual", value)
    
    return df


# 6.
number_moving_average = 'three' # Keep a space at the end

def replace_number_moving_average(df):
    for index, row in df.iterrows():
        # Buscar la expresión regular en la penúltima o última columna
        if pd.notnull(row.iloc[-1]) and re.search(r'(\d\s*-)', str(row.iloc[-1])):
            df.at[index, df.columns[-1]] = re.sub(r'(\d\s*-)', f'{number_moving_average}-', str(row.iloc[-1]))
        #elif pd.notnull(row.iloc[-2]) and re.search(r'(\d\s*-)', str(row.iloc[-2])):
        #   df.at[index, df.columns[-2]] = re.sub(r'(\d\s*-)', f'{number_moving_average}-', str(row.iloc[-2]))
    return df


# 7.
def replace_var_perc_last_columns(df):
    # Expresión regular para buscar "Var. %" o "Var.%"
    regex = re.compile(r'(Var\. ?%)(.*)')

    # Iterar sobre las filas del dataframe
    for index, row in df.iterrows():
        # Verificar si el valor en la penúltima columna es una cadena no nula
        if isinstance(row.iloc[-2], str) and regex.search(row.iloc[-2]):
            # Realizar el reemplazo al final del valor de la penúltima columna
            replaced_text = regex.sub(r'\2 percent change', row.iloc[-2])
            df.at[index, df.columns[-2]] = replaced_text.strip()
        
        # Verificar si el valor en la última columna es una cadena no nula
        if isinstance(row.iloc[-1], str) and regex.search(row.iloc[-1]):
            # Realizar el reemplazo al final del valor de la última columna
            replaced_text = regex.sub(r'\2 percent change', row.iloc[-1])
            df.at[index, df.columns[-1]] = replaced_text.strip()
    
    return df

# 8. Función para buscar y reemplazar en la segunda fila del DataFrame
def replace_first_dot(df):
    second_row = df.iloc[1]  # Segunda fila del DataFrame
    
    # Verificar si al menos una observación cumple con el patrón
    if any(isinstance(cell, str) and re.match(r'^\w+\.\s?\w+', cell) for cell in second_row):
        for col in df.columns:
            if isinstance(second_row[col], str):  # Verificar si el valor es una cadena
                if re.match(r'^\w+\.\s?\w+', second_row[col]):  # Verificar si cumple con el patrón Xxx.Xxx o Xxx. Xxx.
                    df.at[1, col] = re.sub(r'(\w+)\.(\s?\w+)', r'\1-\2', second_row[col], count=1)  # Reemplazar solo el primer punto
    return df

# 9.
def drop_rare_caracter_row(df):
    # Buscar el caracter solitario "}" en cada fila y obtener un booleano para cada fila
    rare_caracter_row = df.apply(lambda row: '}' in row.values, axis=1)
    
    # Filtrar el DataFrame para eliminar las filas con el caracter solitario "}"
    df = df[~rare_caracter_row]
    
    return df

# 10.
def split_column_by_pattern(df):
    # Iteramos sobre las columnas del dataframe
    for col in df.columns:
        # Verificamos si la segunda fila de la columna contiene el patrón
        if re.match(r'^[A-Z][a-z]+\.?\s[A-Z][a-z]+\.?$', str(df.iloc[1][col])):
            # Realizamos el split de la columna usando como criterio el espacio
            split_values = df[col].str.split(expand=True)
            # Guardamos los primeros valores en la columna original
            df[col] = split_values[0]
            # Guardamos los segundos valores en una nueva columna con el sufijo "_split"
            new_col_name = col + '_split'
            df.insert(df.columns.get_loc(col) + 1, new_col_name, split_values[1])
    return df

### Por NS

### Exclusiva Tabla 2

In [ ]:
# 1.
def separate_years(df):
    df = df.copy()  # Create a copy of the DataFrame to avoid SettingWithCopyWarning
    if isinstance(df.iloc[0, -2], str) and len(df.iloc[0, -2].split()) == 2:
        years = df.iloc[0, -2].split()
        if all(len(year) == 4 for year in years):
            second_year = years[1]
            df.iloc[0, -2] = years[0]
            df.insert(len(df.columns) - 1, 'new_column', [second_year] + [None] * (len(df) - 1))
    return df

# 2.
def find_roman_numerals(text):
    pattern = r'\b(?:I{1,3}|IV|V|VI{0,3}|IX|X)\b'
    matches = re.findall(pattern, text)
    return matches

# 3.
def relocate_roman_numerals(df):
    numeros_romanos = find_roman_numerals(df.iloc[2, -1])
    if numeros_romanos:
        original_text = df.iloc[2, -1]
        for roman_numeral in numeros_romanos:
            original_text = original_text.replace(roman_numeral, '').strip()
        df.iloc[2, -1] = original_text
        df.at[2, 'new_column'] = ', '.join(numeros_romanos)
        df.iloc[2, -1] = np.nan
    return df

# 4.
def extract_mixed_values(df):
    df = df.copy()  # Se crea una copia del DataFrame para evitar SettingWithCopyWarning
    regex_pattern = r'(-?\d+,\d [a-zA-Z\s]+)'
    for index, row in df.iterrows():
        antepenultima_obs = row.iloc[-3]
        penultima_obs = row.iloc[-2]

        if isinstance(antepenultima_obs, str) and pd.notnull(antepenultima_obs):
            match = re.search(regex_pattern, antepenultima_obs)
            if match:
                parte_extraida = match.group(0)
                if pd.isna(penultima_obs) or pd.isnull(penultima_obs):
                    df.iloc[index, -2] = parte_extraida
                    antepenultima_obs = re.sub(regex_pattern, '', antepenultima_obs).strip()
                    df.iloc[index, -3] = antepenultima_obs
    return df

# 5.
def replace_first_row_nan(df):
    for col in df.columns:
        if pd.isna(df.iloc[0][col]):
            df.iloc[0, df.columns.get_loc(col)] = col
    return df

# 6. 
def split_values(df):
    columna_a_expandir = df.columns[-3]
    nuevas_columnas = df[columna_a_expandir].str.split(expand=True)
    nuevas_columnas.columns = [f'{columna_a_expandir}_{i+1}' for i in range(nuevas_columnas.shape[1])]
    posicion_insercion = len(df.columns) - 2
    for col in reversed(nuevas_columnas.columns):
        df.insert(posicion_insercion, col, nuevas_columnas[col])
    df.drop(columns=[columna_a_expandir], inplace=True)
    return df


# 7.
def roman_arabic(df):
    primera_fila = df.iloc[0]
    def convert_roman_number(numero):
        try:
            return str(roman.fromRoman(numero))
        except roman.InvalidRomanNumeralError:
            return numero

    primera_fila_convertida = []
    for valor in primera_fila:
        if isinstance(valor, str) and not pd.isna(valor):
            primera_fila_convertida.append(convert_roman_number(valor))
        else:
            primera_fila_convertida.append(valor)

    df.iloc[0] = primera_fila_convertida
    return df

# 8.
def fix_duplicates(df):
    fila_segunda = df.iloc[0].copy()
    prev_num = None
    first_one_index = None

    for i, num in enumerate(fila_segunda):
        try:
            num = int(num)
            prev_num = int(prev_num) if prev_num is not None else None

            if num == prev_num:
                if num == 1:
                    if first_one_index is None:
                        first_one_index = i - 1
                    next_num = int(fila_segunda[i - 1]) + 1
                    for j in range(i, len(fila_segunda)):
                        if fila_segunda.iloc[j].isdigit():
                            fila_segunda.iloc[j] = str(next_num)
                            next_num += 1
                elif i - 1 >= 0:
                    fila_segunda.iloc[i] = str(int(fila_segunda.iloc[i - 1]) + 1)

            prev_num = num
        except ValueError:
            pass

    df.iloc[0] = fila_segunda
    return df